In [ ]:
# !pip install openai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = "localhost:11434"

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:10])

## Generate query from prompt

In [ ]:
import requests
import json

def generate_query_from_promt(prompt):
    url = "http://localhost:11434/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    data = {
        "model": "llama3.2:1b",
        "messages": [
            {
                "role": "user",
                "content": f"Extract 2-3 keywords from this question for vector database search: {prompt}\nReturn only the keywords separated by spaces, no other text:"
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Two-step RAG

In [ ]:
def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. "
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wiki")

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("\n=== Generated Response ===")
    print(response.generative.text)

    print("\n=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
# two_step_rag
two_step_rag("Please provide an explanation at a highschool level. How do airplanes fly?")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")

## Close the client

In [ ]:
client.close()